In [97]:
import os
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model,
)
import pandas as pd
import numpy as np

import IPython.display
import vega

In [98]:
MIRA_PATH = "../../../test/models/april_ensemble_demo/"
SAVE_PATH = "../../../notebook/integration_demo/"
filename = "BIOMD0000000955_template_model.json"
filename = os.path.join(MIRA_PATH, filename)
filename

'../../../test/models/april_ensemble_demo/BIOMD0000000955_template_model.json'

## load_and_sample_petri_model

In [99]:
num_samples = 50
timepoints = [float(x) for x in range(100)]

# Run sampling
samples = load_and_sample_petri_model(
    filename, num_samples, timepoints=timepoints, add_uncertainty=True
)

# Save results
samples.to_csv(os.path.join(SAVE_PATH, "results_petri/sample_results.csv"))

In [100]:
# List of tuples of the form `(time, observation_dict)` where each `observation_dict` is of the form `{observable_name: observable_value}`.
# Once we get a data file format we're very happy to handle the csv -> this format processing.
# Note: As you can see here, not every variable must be observed at every timepoint.
data = [
    (1.1, {"Susceptible": 0.9, "Infected": 0.1}),
    (2.2, {"Susceptible": 0.8}),
]
num_samples = 50

# Run the calibration and sampling
calibrated_samples = load_and_calibrate_and_sample_petri_model(
    filename,
    data,
    num_samples,
    timepoints=timepoints,
    add_uncertainty=True,
    verbose=True,
)

# Save results
calibrated_samples.to_csv(
    os.path.join(SAVE_PATH, "results_petri/calibrated_sample_results.csv")
)

iteration 0: loss = 68.03353643417358
iteration 25: loss = 58.66682147979736
iteration 50: loss = 46.86546778678894
iteration 75: loss = 37.668776512145996
iteration 100: loss = 34.49097228050232
iteration 125: loss = 34.5837721824646
iteration 150: loss = 32.803762435913086
iteration 175: loss = 33.121171951293945
iteration 200: loss = 31.79244041442871
iteration 225: loss = 31.333812713623047
iteration 250: loss = 30.16618537902832
iteration 275: loss = 30.937373161315918
iteration 300: loss = 29.354808568954468
iteration 325: loss = 30.967003345489502
iteration 350: loss = 29.694010257720947
iteration 375: loss = 31.333956718444824
iteration 400: loss = 29.391404151916504
iteration 425: loss = 29.992532968521118
iteration 450: loss = 28.491435766220093
iteration 475: loss = 29.560335159301758
iteration 500: loss = 30.389920949935913
iteration 525: loss = 30.355836629867554
iteration 550: loss = 29.685734748840332
iteration 575: loss = 29.61224603652954
iteration 600: loss = 27.70932

## load_and_calibrate_and_sample_petri_model

In [101]:
## take calibrated and normal dataframes and return 5% and 95%

def plot_predictive(prediction, quantiles):
    # convert series of list to numpy array, then take quantiles
    I_low = np.quantile(np.stack(prediction.values), quantiles[0], axis = 0)
    I_up = np.quantile(np.stack(prediction.values), quantiles[1], axis = 0)
    I_med = np.quantile(np.stack(prediction.values), quantiles[2], axis = 0)

    return I_low, I_up, I_med

# get the time point, medium and quantile values for each sol column
def create_calibration_data(file_path_calibrate_df ="results_petri/calibrated_sample_results.csv", \
                file_path_sample_df ="results_petri/sample_results.csv", \
                 file_path = "calibrate.json", quantiles= (.05, .95, .50),\
                    cases_string = "_sol", time_column = 'timepoint_id', sample_column = 'sample_id' ):
    
    calibrate_df = pd.read_csv(os.path.join(SAVE_PATH, file_path_calibrate_df)).sort_values(by=[sample_column, time_column])
    sample_df = pd.read_csv(os.path.join(SAVE_PATH, file_path_sample_df)).sort_values(by=[sample_column, time_column])

    sol_cols = [c for c in calibrate_df.columns if c.lower()[-len(cases_string):] == cases_string]

    new_df = pd.DataFrame()
    length_time = len(calibrate_df[time_column].unique())

    time_list = []
    column_names_list = []
    calibration_list = []
    column_values_y0 = []
    column_values_y1 = []
    column_values_y = []
    column_values_example = []
    # for each sol column add low and high 5% and 95%
    for col in sol_cols:
        # sort by sample_id and timepoint, get list of sample id's in order of time, 
        list_sol_calibrate = calibrate_df.groupby(sample_column)[col].apply(list).reset_index()[col]
        list_sol = sample_df.groupby(sample_column)[col].apply(list).reset_index()[col]

        # get plot predictive
        I_low_calibrate, I_up_calibrate, I_med_calibrate = plot_predictive(list_sol_calibrate, quantiles)
        column_names_list.extend([col ] * length_time)
        calibration_list.extend([False] * length_time)
        # add values
        column_values_y0.extend(I_low_calibrate.tolist())
        column_values_y1.extend(I_up_calibrate.tolist())
        column_values_y.extend(I_med_calibrate.tolist())
        time_list.extend(calibrate_df[time_column].unique())

        # get plot predictive
        I_low, I_up, I_med = plot_predictive(list_sol, quantiles)
        column_names_list.extend([col] * length_time)
        calibration_list.extend([True] * length_time)

        # add values
        column_values_y0.extend(I_low.tolist())
        column_values_y1.extend(I_up.tolist())
        column_values_y.extend(I_med.tolist())
        time_list.extend(calibrate_df[time_column].unique())
        


    new_df['time'] = time_list
    new_df['column_names'] = column_names_list
    new_df['calibration'] = calibration_list
    new_df['y'] = column_values_y
    new_df['y1'] = column_values_y1
    new_df['y0'] = column_values_y0
    return new_df
# json in format time sol_low sol_high

cal_data = create_calibration_data()

In [102]:
vega.ipy_display(vega.calibrate_vega(cal_data))

In [103]:
#get vega plot